## Tutorial Python: Modelos Avanzados de Clasificacion

Carguemos los libraries utility

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV

Y ahora los clasificadores que usaremos en este tutorial

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

Set up Cross-Validation (k=10)

In [4]:
k_fold = KFold(n_splits=10)

Importemos los datos

In [5]:
datos = pd.read_csv('/Users/carloseqa/Dropbox/RStudioProjects/PredictiveAnalytics/bwt.csv')
datos.head()

,id,age,race,smoke,ptd,ht,ui,ftv,lwt,lowweight
0,1,19,black,no,no,no,yes,0,182,no
1,2,33,other,no,no,no,no,3,155,no
2,3,20,white,yes,no,no,no,1,105,no
3,4,21,white,yes,no,no,yes,2,108,no
4,5,18,white,yes,no,no,yes,0,107,no


Sklearn solo acepta variables numericas (aun para clasificacion). Es necesario definir dummies para todas las variables categoricas, incluyendo la variable dependiente. Python creara dummies para todas las categorias; una de ellas siempre es redundante. El setting 'drop_first=True' elimina la primera categoria.

In [6]:
datos=pd.get_dummies(datos,drop_first=True)
datos.head()

,id,age,ftv,lwt,race_other,race_white,smoke_yes,ptd_yes,ht_yes,ui_yes,lowweight_yes
0,1,19,0,182,0,0,0,0,0,1,0
1,2,33,3,155,1,0,0,0,0,0,0
2,3,20,1,105,0,1,1,0,0,0,0
3,4,21,2,108,0,1,1,0,0,1,0
4,5,18,0,107,0,1,1,0,0,1,0


Sklearn pide variable dependiente e independientes por separado. Normalicemos desde ahora los datos para uso con SVM

In [7]:
y = datos.lowweight_yes
X = datos.drop(datos.columns[[0,10]],axis=1)
X_scaled = scale(X)
X.head()

,age,ftv,lwt,race_other,race_white,smoke_yes,ptd_yes,ht_yes,ui_yes
0,19,0,182,0,0,0,0,0,1
1,33,3,155,1,0,0,0,0,0
2,20,1,105,0,1,1,0,0,0
3,21,2,108,0,1,1,0,0,1
4,18,0,107,0,1,1,0,0,1


## Random Forest

Ahora instanciemos y estimemos el modelo de Random Forest

In [8]:
modelo4 = RandomForestClassifier(random_state=25,n_estimators=500,max_features=2)
modelo4.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=25, verbose=0, warm_start=False)

In [9]:
scores4= cross_val_score(modelo4, X, y, cv=k_fold, n_jobs=-1)
np.mean(scores4)

0.5640350877192983

In [10]:
print(modelo4.feature_importances_)

[0.27608096 0.10303602 0.33786538 0.03082063 0.03991744 0.05302338
 0.07078219 0.0345712  0.0539028 ]


In [11]:
X.columns

Index(['age', 'ftv', 'lwt', 'race_other', 'race_white', 'smoke_yes', 'ptd_yes',
       'ht_yes', 'ui_yes'],
      dtype='object')

## Gradient Boosting Machine

In [12]:
modelo5 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=1, subsample=0.75)
modelo5.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=1,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=0.75, verbose=0,
              warm_start=False)

In [13]:
scores5= cross_val_score(modelo5, X, y, cv=k_fold, n_jobs=-1)
np.mean(scores5)

0.6116959064327485

## Support Vector Machines

In [20]:
modelo6 = SVC()
modelo6.fit(X_scaled,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [21]:
scores6= cross_val_score(modelo5, X_scaled, y, cv=k_fold, n_jobs=-1)
np.mean(scores6)

0.606140350877193

## Support Vector Machines (con Grid Search)

Ahora hagamos un poco de parameter search. Ustedes pueden ajustar el programa para que considere mas valores y tambien lo pueden aplicar a Random Forest y Gradient Boosting Machine.

In [16]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = SVC()
modelo7=GridSearchCV(svc, parameters)
modelo7.fit(X_scaled,y)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ('linear', 'rbf'), 'C': [1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [19]:
scores7= cross_val_score(modelo7, X_scaled, y, cv=k_fold, n_jobs=-1)
np.mean(scores7)

0.6432748538011696